In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
'BIGQUERY.json')
project_id = 'handy-cell-347207'
client = bigquery.Client(credentials= credentials, project=project_id)

In [2]:
## Querying the table of World Port Index to find all columns
##Make an API request
query= client.query(""" 
SELECT * 
FROM `bigquery-public-data.geo_international_ports.world_port_index`
LIMIT 1000;""")

data=query.result() ##Waits for query to finish
rows=list(data)
print(rows)

[Row(('50922', '50920', 'MERAK MAS TERMINAL', 'ID', -5.916667, 105.983333, '150', None, 'M', 'CN', 'F', None, None, None, None, None, 'J', 'A', 'J', 'J', 0, 'M', None, None, None, None, None, None, True, None, None, None, True, None, None, None, True, None, True, None, None, None, True, None, None, None, None, True, None, None, None, None, True, None, None, None, None, None, None, None, None, None, None, None, True, True, True, None, None, None, None, None, 'POINT(105.983333333333 -5.91666666666667)'), {'index_number': 0, 'region_number': 1, 'port_name': 2, 'country': 3, 'port_latitude': 4, 'port_longitude': 5, 'publication_number': 6, 'chart_number': 7, 'harbor_size': 8, 'harbor_type': 9, 'shelter_afforded': 10, 'entry_tide': 11, 'entry_swell': 12, 'entry_ice': 13, 'entry_other': 14, 'overhead_limit': 15, 'channel_depth': 16, 'anchorage_depth': 17, 'cargo_pier_depth': 18, 'cargo_oil_depth': 19, 'tide_range': 20, 'max_vessel_size': 21, 'good_holding_ground': 22, 'turning_area': 23, 'po

In [3]:
##Answer for Question 1:
##Querying the columns of Jurong Island to find out its longitude and latitude
##Make an API request
query1= client.query("""
SELECT port_latitude, 
       port_longitude, 
FROM `bigquery-public-data.geo_international_ports.world_port_index` 
WHERE port_name= 'JURONG ISLAND';

##Using geospatial functions: ST_GEOGPOINT(longitude, latitude) to find distance from a point
WITH params AS 
  (SELECT ST_GeogPoint(103.733333,1.283333) AS centre_point,
         5 AS max_nearest_ports,
         3000 AS max_dist_km),

distance_from_centre_point AS 
  (SELECT index_number,
          port_name,
          country,
          ST_GeogPoint(port_longitude, port_latitude) AS location,

##ST_Distance to compute the distance in meters
          ST_Distance(ST_GeogPoint(port_longitude, port_latitude), params.centre_point) AS distance_in_meters
  FROM `bigquery-public-data.geo_international_ports.world_port_index`, 
    params

##ST_DWithin to filter the nearest ports within 3000km from Jurong Island
##DWITHIN can do optimizations to compute only distance of ports that is close enough
  WHERE ST_DWithin(ST_GeogPoint(port_longitude, port_latitude), params.centre_point, params.max_dist_km*3000) 
  AND port_name != 'JURONG ISLAND')

SELECT port_name,
       distance_in_meters,
FROM distance_from_centre_point
ORDER BY distance_in_meters
LIMIT 5;
""")

data=query1.result() ##Waits for query to finish
rows=list(data)
print(rows)

[Row(('PULAU BUKOM', 6681.539244071895), {'port_name': 0, 'distance_in_meters': 1}), Row(('PULAU SEBAROK', 11865.507282847706), {'port_name': 0, 'distance_in_meters': 1}), Row(('KEPPEL - (EAST SINGAPORE)', 12969.544857136169), {'port_name': 0, 'distance_in_meters': 1}), Row(('TANJUNG PELEPAS', 21686.673244141813), {'port_name': 0, 'distance_in_meters': 1}), Row(('PULAU SAMBU', 22618.33518201838), {'port_name': 0, 'distance_in_meters': 1})]


In [4]:
##Answer for Question 2:
##Make an API request
query2= client.query("""
SELECT country, COUNT(*) AS port_count
FROM `bigquery-public-data.geo_international_ports.world_port_index` 
WHERE cargo_wharf = true
GROUP BY country
ORDER BY port_count DESC
LIMIT 1;
""")

data=query2.result() ##Waits for query to finish
rows=list(data)
print(rows)

[Row(('US', 583), {'country': 0, 'port_count': 1})]


In [5]:
##Answer for Question 3:
##Make an API request
query3= client.query("""
WITH params AS 
  (SELECT ST_GeogPoint(-38.706256,32.610982) AS centre_point),

distance_from_centre_point AS 
  (SELECT index_number,
          country,
          port_name,
          port_latitude,
          port_longitude, 
          ST_Distance(ST_GeogPoint(port_longitude, port_latitude), params.centre_point) AS distance_in_meters
  FROM `bigquery-public-data.geo_international_ports.world_port_index`, 
    params
  WHERE provisions=true AND water=true AND fuel_oil=true AND diesel=true)

SELECT country,
       port_name, 
       port_latitude,
       port_longitude
FROM distance_from_centre_point
ORDER BY distance_in_meters
LIMIT 5;
""")

data=query3.result() ##Waits for query to finish
rows=list(data)
print(rows)

##Reference: https://towardsdatascience.com/using-bigquerys-new-geospatial-functions-to-interpolate-temperatures-d9363ff19446
##Reference: https://sophiamyang.github.io/DS/dataaccess/bigquery/bigquery.html
##Reference: https://stackoverflow.com/questions/47490789/print-to-view-result-of-bigquery-query

[Row(('PT', 'HORTA', 38.533333, -28.633333), {'country': 0, 'port_name': 1, 'port_latitude': 2, 'port_longitude': 3}), Row(('PT', 'PRAIA DE VITORIA', 38.716667, -27.05), {'country': 0, 'port_name': 1, 'port_latitude': 2, 'port_longitude': 3}), Row(('PT', 'PONTA DELGADA', 37.733333, -25.666667), {'country': 0, 'port_name': 1, 'port_latitude': 2, 'port_longitude': 3}), Row(('PT', 'FUNCHAL', 32.633333, -16.916667), {'country': 0, 'port_name': 1, 'port_latitude': 2, 'port_longitude': 3}), Row(('CA', "ST JOHN'S", 47.566667, -52.7), {'country': 0, 'port_name': 1, 'port_latitude': 2, 'port_longitude': 3})]


In [6]:
##Answer for Question 4a:
## Refactoring is a way to improve the structure of the codes design so that it is clean and simple without altering its functionailty. 
## Refactoring can be done before adding more complex codes to the existing one. 
## Refactoring is an important step as it makes the codes easy to understand and maintain for everyone includiing the new joiners in the team.  


##Answer for Question 4b:
## Testing is to find out if something works as intended that is set out in the business requirements.
## Testing is done frequently in the initial stages after gathering business requirements and before the final product is rolled out.
## Testing is important as it determines if the final product that is rolled out conforms to standards and business requirements and is of good quality.